In [1]:
from sklearn.ensemble import RandomForestClassifier
#import deepchem as dc
import numpy as np
import pandas as pd
import tempfile
#import chemprop
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import backend as K
from tensorflow.keras import initializers
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from sklearn.linear_model import LogisticRegression
import xgboost
import os
from sklearn.svm import SVC
from tensorflow.keras.callbacks import EarlyStopping
import joblib

In [2]:
import matplotlib.pyplot as plt 
import math
from scipy.stats import ttest_ind,linregress;
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve
from sklearn.metrics import matthews_corrcoef
#from sklearn.metrics import recall_score
#from sklearn.metrics import precision_score
from sklearn.metrics import auc, mean_absolute_error, mean_squared_error, precision_recall_curve, \
r2_score,roc_auc_score, accuracy_score, log_loss
from sklearn.metrics import confusion_matrix,cohen_kappa_score
from sklearn.metrics import f1_score,confusion_matrix

# generate AtompairFP with chiral

In [3]:
from rdkit.Chem.AtomPairs import Pairs
from rdkit.Chem import DataStructs
from rdkit import Chem
def GetAtomPairFPs(mol, nBits = 2048, binary = True, includeChirality=False):
    '''
    atompairs fingerprints
    '''
    fp = Pairs.GetHashedAtomPairFingerprint(mol, nBits = nBits, includeChirality=includeChirality)
    if binary:
        arr = np.zeros((0,),  dtype=np.bool)
    else:
        arr = np.zeros((0,),  dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, arr)
    arr = arr.astype(np.int8)
    return arr

In [4]:
def gen_AtomPairFP_file_with_chiral(x_csv):
    x_base = x_csv.replace('.csv','')
    df = pd.read_csv(x_csv)
    smiles_col = 'smiles'
    smiles_list = df[smiles_col].values
    csv_name = f'./fp/{x_base}_AtomPairFPC.csv'
    f=open(csv_name, 'w')
    #f.write('smiles,'+ ','.join(['MACCS{:0>3}'.format(x) for x in range(167)]) +'\n')  ## 167
    f.write(','.join(['AtomPairFPC{}'.format(x) for x in range(2048)]) +'\n')  
    for smi in smiles_list:
        mol = Chem.MolFromSmiles(smi)
        key = GetAtomPairFPs(mol, nBits = 2048, binary = True,includeChirality=True)
        #f.write(smi+',' + ','.join([str(x) for x in key])+'\n')
        f.write(','.join([str(x) for x in key])+'\n')
    f.close()
    print('done {}'.format(csv_name))

In [5]:
csv_list = ['Main.csv','Ext.csv']
for csv in csv_list:
    gen_AtomPairFP_file_with_chiral(csv)

/tmp/ipykernel_1329908/1019570494.py:10: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  arr = np.zeros((0,),  dtype=np.bool)


done ./fp/Main_AtomPairFPC.csv
done ./fp/Ext_AtomPairFPC.csv


In [6]:
def ROC_AUC(y_true, y_score):
	auc = roc_auc_score(y_true, y_score)
	return auc

## morganC and AtomPairFPC represent the MorganFP and AtomPairFP incorprated with Chiral parameter

In [7]:
fingerprint_list = ['morganC','AtomPairFPC'] #['MorganFP', 'morganC']

In [8]:
def get_Xye_for_one_fingerprint(feature):
    #feature='AtomPairFP'
    csv_file = "Main.csv"
    df = pd.read_csv(csv_file)
    smiles_col = df.columns[0]
    values_col = df.columns[1]

    csv_feat_file = f"fp/Main_{feature}.csv"
    dff = pd.read_csv(csv_feat_file)

    print('values_col = ',values_col)
    #feats_col = df.columns[2:]

    MASK = -1
    #y = df[values_col].astype('int').fillna(MASK).values
    y = df[values_col].astype('int').fillna(MASK).values
    #X = df[feats_col].values
    X = dff.values
    n_feats = len(dff.columns)

    #read external set
    csv_file_e = "Ext.csv"
    df_e = pd.read_csv(csv_file_e)
    smiles_col = df_e.columns[0]
    values_col = df_e.columns[1]
    csv_feat_file_e = f"fp/Ext_{feature}.csv"
    df_ef = pd.read_csv(csv_feat_file_e)

    MASK = -1
    y_e = df_e[values_col].astype('int').fillna(MASK).values
    X_e = df_ef.values
    return X,y,X_e,y_e

In [10]:
# seem the keras model can only run once inside a function, otherwise will get GPU errors.
def run_one_keras_fcnn(feature,i):   
    model = Sequential()
    model.add(Dense(800, input_shape=(n_feats,), activation='relu'),)
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
    #print(model.summary)
    pklf = f'./rand_MorganFP/get_split/split_indices_fold{i}.pckl'
    train_idx, valid_idx, test_idx = pd.read_pickle(pklf)
    print(len(train_idx), len(valid_idx), len(test_idx))
    #get X and Y (main set and external set)
    X_T = X[train_idx];y_T = y[train_idx]
    X_valid = X[valid_idx];y_valid = y[valid_idx]
    X_test = X[test_idx];y_test = y[test_idx]     
    y_T_2dim=y_T[:,np.newaxis]  ### change shape from(2769,) to (2769, 1)
    y_valid_2dim = y_valid[:,np.newaxis]
    print(y_T_2dim.shape)
    #model.fit(X_T, y_T_2dim,batch_size=4096,verbose=0, epochs=100)
    ### using early stop by monitor val_loss of X_valid, y_valid_2dim
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    model.fit(X_T, y_T_2dim, batch_size=4096, verbose=0, epochs=100, 
              validation_data=(X_valid, y_valid_2dim), callbacks=[early_stopping])    
    ## prediction
    y_prob_T = model.predict(X_T) 
    train_roc_auc = ROC_AUC(y_T,y_prob_T[:, 0])
    print('train_roc_auc = ', train_roc_auc )
    y_prob_valid=model.predict(X_valid)  ## no need other parameters here
    valid_roc_auc = ROC_AUC(y_valid,y_prob_valid[:, 0])
    y_prob_test=model.predict(X_test)  
    test_roc_auc = ROC_AUC(y_test,y_prob_test[:, 0])
    print('test_roc_auc, valid_roc_auc = ', test_roc_auc, valid_roc_auc)
    y_prob_e=model.predict(X_e)  ## no need other parameters here
    e_roc_auc = ROC_AUC(y_e,y_prob_e[:, 0])
    print('e_roc_auc = ', e_roc_auc)    
    ## save predition
    if not os.path.exists('fcnn'):os.mkdir('fcnn')
    pd.DataFrame(y_prob_T[:,0],columns=['prob']).to_csv(f'fcnn/p_train_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_valid[:, 0],columns=['prob']).to_csv(f'fcnn/p_val_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_test[:,0],columns=['prob']).to_csv(f'fcnn/p_test_{feature}_fold{i}.csv',index=False)
    pd.DataFrame(y_prob_e[:,0],columns=['prob']).to_csv(f'fcnn/p_Ext_{feature}_fold{i}.csv',index=False) 
    ## save model
    model_save_name = f'fcnn/model_{feature}_fold{i}.h5'
    model.save(model_save_name)    
    model = ''
    ## empty gpu memory
    import torch
    torch.cuda.empty_cache()
    return   

# Run fcnn on both morganC and AtomPairFPC

In [11]:
os.environ["CUDA_VISIBLE_DEVICES"]="2"
num_folds = 5
for feature in fingerprint_list:
    csv_feat_file = f"fp/Main_{feature}.csv"
    tdf = pd.read_csv(csv_feat_file,nrows=3)
    n_feats = len(tdf.columns)
    print(f'processing feat: {feature}')
    X,y,X_e,y_e = get_Xye_for_one_fingerprint(feature)
    for i in range(num_folds): 
        run_one_keras_fcnn(feature,i)

processing feat: morganC
values_col =  label


2023-02-10 20:08:27.930514: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 20:08:28.584289: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 8037 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:81:00.0, compute capability: 8.6


2769 346 347
(2769, 1)


2023-02-10 20:08:30.282678: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9971989085045687
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9527450980392156 0.9332634521313767
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.8495653023346683
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9986737300645285
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.903406862745098 0.9501952602578506
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.8412295268796197
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 1ms/step
train_roc_auc =  0.9966637186731289
11/11 [==============================] - 0s 1ms/step
test_roc_auc, valid_roc_auc =  0.9598845598845599 0.9471383147853736
14/14 [==============================] - 0s 1ms/step
e_roc_auc =  0.8343264628309076
2769 346 347
(2769, 1)
87/87 [==============================] - 0s 

In [12]:
h1_list = ['r2_score','pearson_r2','rmse','mse','mae','roc','prc','p-stat','Recall',
'Precision','f1','BA','accuracy', 'TN', 'FP', 'FN','TP','SP','SE','NPV','MCC']

In [13]:
def polyfit(x, y, degree):
	#coeffs = numpy.polyfit(x, y, degree) # Polynomial Coefficients
	correlation = numpy.corrcoef(x, y)[0,1]
	return correlation**2

def calc_class_roc_prc(y_true,y_pred,pos_label=1):
	#print('y_true =', y_true)
	#y_ = [y for y in y_true if y not in [0,1]]
	#print('y_ = ',y_)   ## -9223372036854775808, must be NaN
	x_roc=roc_auc_score(y_true,y_pred)
	if pos_label==0:x_roc = 1 - x_roc		
	x_precision, x_recall, x_thresholds = precision_recall_curve(y_true,y_pred)
	x_prc = auc(x_recall, x_precision)
	return {'roc':x_roc,'prc':x_prc}

def calc_class_other_stats_with_threshold(y_true, y_pred, threshold=0.5):
	ind0=np.where(y_true==0)[0];ind1=np.where(y_true==1)[0]
	y_pred0=y_pred[ind0];y_pred1=y_pred[ind1];
	p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
	#print('p_stat = ', p_0_1)
	y_hard_pred = [1 if p > threshold else 0 for p in y_pred] # binary prediction
	#x_cohen_kappa = cohen_kappa_score(y_true, y_pred, weights='linear')	
	### not understand, can problem, Can't handle mix of binary and continuous
	x_cohen_kappa = cohen_kappa_score(y_true, y_hard_pred, weights='linear')
	## True and false values
	TN, FP, FN, TP = confusion_matrix(y_true, y_hard_pred).ravel()	
	#TP = sum((y_true == 1) & (y_hard_pred == 1))
	#TN = sum((y_true == 0) & (y_hard_pred == 0))
	#FN = sum((y_true == 1) & (y_hard_pred == 0))
	#FP = sum((y_true == 0) & (y_hard_pred == 1))
	# SE (Sensitivity), hit rate, recall, or true positive rate
	x_Recall = TP/(TP+FN)
	SE=x_Recall
	# Precision or positive predictive value
	x_Precision = TP/(TP+FP)
	#f1_score = (2*x_Precision*x_Recall)/ (x_Precision + x_Recall)
	f1 = f1_score(y_true, y_hard_pred)
	# CCR (Correct classification rate), BA (balanced accuracy)
	# Specificity or true negative rate
	SP = TN/(TN+FP) 
	x_BA = (SE + SP)/2
	x_accuracy = accuracy_score(y_true, y_hard_pred)
	# Negative predictive value
	NPV = TN/(TN+FN)
	x_MCC = matthews_corrcoef(y_true, y_hard_pred)
	other_stats = {'p_stat':p_0_1,'Recall':x_Recall,'Precision':x_Precision, 'f1':f1,'BA':x_BA,
	'accuracy':x_accuracy, 'TN':TN, 'FP':FP, 'FN':FN,'TP':TP,'SP':SP,'SE':SE,
	'NPV':NPV,'MCC':x_MCC,'cohen_kappa':x_cohen_kappa}  # 
	return other_stats

In [14]:
def calc_ef_threholds(y_true,y_pred,ef_threholds=[0.01,0.05,0.1]):
	# y_true,y_pred are np.array type
	df = pd.DataFrame()   ## merge  y_true,y_pred as df 's  two cols
	df['y_true'] = y_true;df['y_pred'] = y_pred
	n_actives = len(df[df['y_true']==1].index)
	n_total = len(df.index)
	random_rate = n_actives/ n_total
	df.sort_values(by='y_pred',ascending=False,inplace=True)
	EFs = {}
	for ef_threhold in ef_threholds:
		screen_range = int(np.ceil(n_total * ef_threhold))
		screen_rate = sum(df['y_true'][:screen_range]) / screen_range
		#print('n_actives,n_total = ',n_actives,n_total)
		print('random_rate,screen_range,screen_rate = ', random_rate,screen_range,screen_rate)
		EF = screen_rate / random_rate
		EF_name = 'EF_{}'.format(ef_threhold)
		#print('{} = {}'.format(EF_name,EF))
		EFs.update({EF_name:EF})
	return EFs

In [15]:
def list_stat_for_class(true_file,pred_file,pos_label,ef_threholds):
    t_df = pd.read_csv(true_file)
    p_df = pd.read_csv(pred_file)
    #t_values = t_df[label_col].values
    t_values = t_df.iloc[:,1].values.astype(int)
    if model == 'DMPNN': p_values = p_df.iloc[:,1].values
    else: p_values = p_df.iloc[:,0].values
    ind = np.where((t_values==0) | (t_values==1))[0]	 ### to remove those NaN label
    #print('ind = ',ind)
    y_true = t_values
    [ind]
    y_pred = p_values[ind]
    print('y_true[:10], y_pred[:10] =',y_true[:10], y_pred[:10])
    dic = {}
    dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
    dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
    dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
    w_line= pred_file +',' + ','.join([str(dic[x]) for x in header])+'\n'
    f = open(eva_csv_name, 'a'); f.write(w_line); f.close()
    return 

In [16]:
fingerprint_list = ['MorganFP', 'morganC']

models = ['fcnn',]
num_folds = 5
#true_file = 'hide/rand_check/fold_0/train_full.csv'
b_list = ['train','val','test','Ext']   ## DMPNN use this

true_file_pat = 'rand_/fold_{i}/{b}_full.csv' 
true_file_e = 'Ext.csv'

pred_file_pat = '{model}/p_{b}_{feature}_fold{i}.csv'  # include DMPNN

In [17]:
efs=ef_threholds=[0.01,0.02,0.05]
thre=threshold=0.5
pos_label = 1

## write csv head

In [18]:
eva_csv_name = 'eva_Morgan_chiral.csv'
dic = {}
y_true,y_pred =np.array([1,0]), np.array([0.9,0.1])
dic.update(calc_class_roc_prc(y_true,y_pred,pos_label))
dic.update(calc_ef_threholds(y_true,y_pred,ef_threholds))
dic.update(calc_class_other_stats_with_threshold(y_true, y_pred,threshold))
header=list(dic.keys()); #header=sorted(header)
tmp = [x for x in h1_list if x in header] + [x for x in header if x not in h1_list]
header = tmp
print('header=',header)
f = open(eva_csv_name, 'w');f.write('col_names,' + ','.join(header)+'\n');f.close()

random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
random_rate,screen_range,screen_rate =  0.5 1 1.0
header= ['roc', 'prc', 'Recall', 'Precision', 'f1', 'BA', 'accuracy', 'TN', 'FP', 'FN', 'TP', 'SP', 'SE', 'NPV', 'MCC', 'EF_0.01', 'EF_0.02', 'EF_0.05', 'p_stat', 'cohen_kappa']


/tmp/ipykernel_1329908/448253304.py:19: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  p_0_1 = ttest_ind(y_pred0, y_pred1)[1]	 #"%.2e"%
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: divide by zero encountered in true_divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/home/phzd/miniconda3/envs/chemprop1.4.0/lib/python3.8/site-packages/scipy/stats/_stats_py.py:1250: RuntimeWarning: invalid value encountered in double_scalars
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


## eva metrics

In [20]:
fps= ['MorganFP', 'morganC','AtomPairFP', 'AtomPairFPC']
for feature in fps:
    for model in models:
        print(f'processing feature: {feature}; model: {model}')
        for b in b_list: 
            for i in range(num_folds):
                if b=='Ext': true_file = 'Ext.csv'
                else: true_file = f'rand_/fold_{i}/{b}_full.csv'             
                pred_file = f'{model}/p_{b}_{feature}_fold{i}.csv'  
                # except DMPNN, other model use this
                print('true_file,pred_file = ',true_file,pred_file)
                list_stat_for_class(true_file,pred_file,pos_label,ef_threholds)  

processing feature: MorganFP; model: fcnn
true_file,pred_file =  rand_/fold_0/train_full.csv fcnn/p_train_MorganFP_fold0.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 0 1 1 1] [0.9875788  0.9344152  0.7041694  0.9953774  0.00909553 0.9886895
 0.65400046 0.99416935 0.99883956 0.9821354 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_/fold_1/train_full.csv fcnn/p_train_MorganFP_fold1.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 1 1 1 0] [0.59657246 0.9986696  0.9852417  0.9402775  0.9805925  0.9893783
 0.99451065 0.9846038  0.99836975 0.02771047]
random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_/fold_2/train_full.csv fcnn/p_train_MorganFP

random_rate,screen_range,screen_rate =  0.7771758757674251 28 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 56 1.0
random_rate,screen_range,screen_rate =  0.7771758757674251 139 1.0
true_file,pred_file =  rand_/fold_2/train_full.csv fcnn/p_train_morganC_fold2.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 1 0 1 1 1] [0.09543221 0.9955777  0.999681   0.9999453  0.99521095 0.9981433
 0.19484796 0.92763245 0.988674   0.9778511 ]
random_rate,screen_range,screen_rate =  0.7804261466233298 28 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 56 1.0
random_rate,screen_range,screen_rate =  0.7804261466233298 139 1.0
true_file,pred_file =  rand_/fold_3/train_full.csv fcnn/p_train_morganC_fold3.csv
y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.1561273  0.9784056  0.93342996 0.99757665 0.98447233 0.17456433
 0.9918699  0.9900535  0.99920636 0.5579547 ]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318

y_true[:10], y_pred[:10] = [0 1 1 1 1 0 1 1 1 0] [0.20242871 0.9995177  0.82471794 0.99527836 0.9946131  0.1378944
 0.9968077  0.98615116 0.9953803  0.71233565]
random_rate,screen_range,screen_rate =  0.7822318526543879 28 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 56 1.0
random_rate,screen_range,screen_rate =  0.7822318526543879 139 1.0
true_file,pred_file =  rand_/fold_4/train_full.csv fcnn/p_train_AtomPairFP_fold4.csv
y_true[:10], y_pred[:10] = [1 1 1 1 0 1 1 0 0 0] [0.9633749  0.97825694 0.97952545 0.9960198  0.04900435 0.758726
 0.9963684  0.05465533 0.33284244 0.04020888]
random_rate,screen_range,screen_rate =  0.7833152762730228 28 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 56 1.0
random_rate,screen_range,screen_rate =  0.7833152762730228 139 1.0
true_file,pred_file =  rand_/fold_0/val_full.csv fcnn/p_val_AtomPairFP_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.9991929  0.3533959  0.96908903 0.93397295 0.99989617 0.9745739


true_file,pred_file =  rand_/fold_0/val_full.csv fcnn/p_val_AtomPairFPC_fold0.csv
y_true[:10], y_pred[:10] = [1 0 1 1 1 1 1 1 1 1] [0.9982474  0.31756833 0.9713066  0.8925144  0.99997103 0.9740879
 0.998381   0.9953295  0.5790977  0.9994784 ]
random_rate,screen_range,screen_rate =  0.7658959537572254 4 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 7 1.0
random_rate,screen_range,screen_rate =  0.7658959537572254 18 1.0
true_file,pred_file =  rand_/fold_1/val_full.csv fcnn/p_val_AtomPairFPC_fold1.csv
y_true[:10], y_pred[:10] = [1 0 0 1 1 0 0 1 1 0] [0.91817176 0.00187588 0.9000076  0.992167   0.99999833 0.14802855
 0.0080508  0.99918395 0.9984114  0.34638256]
random_rate,screen_range,screen_rate =  0.8063583815028902 4 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 7 1.0
random_rate,screen_range,screen_rate =  0.8063583815028902 18 1.0
true_file,pred_file =  rand_/fold_2/val_full.csv fcnn/p_val_AtomPairFPC_fold2.csv
y_true[:10], y_pred[:10] = [1 1 0 0 1 1